<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 803ms/step - loss: 0.6547 - accuracy: 0.6071 - val_loss: 0.6825 - val_accuracy: 0.5220
Epoch 2/10
26/26 [==============================] - 20s 769ms/step - loss: 0.5998 - accuracy: 0.6898 - val_loss: 0.5909 - val_accuracy: 0.6683
Epoch 3/10
26/26 [==============================] - 20s 765ms/step - loss: 0.5469 - accuracy: 0.7397 - val_loss: 0.5465 - val_accuracy: 0.6878
Epoch 4/10
26/26 [==============================] - 20s 762ms/step - loss: 0.5106 - accuracy: 0.7470 - val_loss: 0.5340 - val_accuracy: 0.7366
Epoch 5/10
26/26 [==============================] - 20s 761ms/step - loss: 0.4550 - accuracy: 0.8017 - val_loss: 0.4388 - val_accuracy: 0.8146
Epoch 6/10
26/26 [==============================] - 20s 762ms/step - loss: 0.4330 - accuracy: 0.7981 - val_loss: 0.4317 - val_accuracy: 0.8488
Epoch 7/10
26/26 [==============================] - 21s 815ms/step - loss: 0.4145 - accuracy: 0.8163 - val_loss: 0.3567 - val_accuracy: 0.8537

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.68.h5
26/26 - 22s - loss: 0.6657 - accuracy: 0.5620 - val_loss: 0.6824 - val_accuracy: 0.4780
Epoch 2/5

Epoch 00002: saving model to weights.02-0.81.h5
26/26 - 22s - loss: 0.5943 - accuracy: 0.7409 - val_loss: 0.8117 - val_accuracy: 0.4488
Epoch 3/5

Epoch 00003: saving model to weights.03-0.49.h5
26/26 - 23s - loss: 0.5445 - accuracy: 0.7543 - val_loss: 0.4872 - val_accuracy: 0.8488
Epoch 4/5

Epoch 00004: saving model to weights.04-0.58.h5
26/26 - 20s - loss: 0.5005 - accuracy: 0.7835 - val_loss: 0.5753 - val_accuracy: 0.6585
Epoch 5/5

Epoch 00005: saving model to weights.05-0.36.h5
26/26 - 20s - loss: 0.4315 - accuracy: 0.8163 - val_loss: 0.3591 - val_accuracy: 0.8829


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 24s - loss: 0.6800 - accuracy: 0.5693 - val_loss: 0.6690 - val_accuracy: 0.7756


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6793 - accuracy: 0.5645 - val_loss: 0.7194 - val_accuracy: 0.4341
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6563 - accuracy: 0.6119 - val_loss: 0.6453 - val_accuracy: 0.6439


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6567 - accuracy: 0.6314 - val_loss: 0.6087 - val_accuracy: 0.7268
Epoch 2/50
26/26 - 20s - loss: 0.5832 - accuracy: 0.7068 - val_loss: 0.5383 - val_accuracy: 0.8195
Epoch 3/50
26/26 - 21s - loss: 0.5119 - accuracy: 0.7701 - val_loss: 0.5072 - val_accuracy: 0.7610
Epoch 4/50
26/26 - 20s - loss: 0.4522 - accuracy: 0.8017 - val_loss: 0.5158 - val_accuracy: 0.7024
Epoch 5/50
26/26 - 20s - loss: 0.3790 - accuracy: 0.8613 - val_loss: 0.3189 - val_accuracy: 0.9122
Epoch 6/50
26/26 - 20s - loss: 0.2993 - accuracy: 0.9002 - val_loss: 0.3136 - val_accuracy: 0.8878
Epoch 7/50
26/26 - 20s - loss: 0.2873 - accuracy: 0.8893 - val_loss: 0.2666 - val_accuracy: 0.8976
Epoch 8/50
26/26 - 20s - loss: 0.2087 - accuracy: 0.9270 - val_loss: 0.1564 - val_accuracy: 0.9659
Epoch 9/50
26/26 - 19s - loss: 0.1895 - accuracy: 0.9404 - val_loss: 0.1701 - val_accuracy: 0.9512
Epoch 10/50
26/26 - 20s - loss: 0.1587 - accuracy: 0.9550 - val_loss: 0.1389 - val_accuracy: 0.9561
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 783ms/step - loss: 0.6594 - accuracy: 0.6204 - val_loss: 0.6427 - val_accuracy: 0.7610
Epoch 2/5
26/26 [==============================] - 20s 764ms/step - loss: 0.6254 - accuracy: 0.6582 - val_loss: 0.6029 - val_accuracy: 0.7659
Epoch 3/5
26/26 [==============================] - 20s 777ms/step - loss: 0.5922 - accuracy: 0.6776 - val_loss: 0.6821 - val_accuracy: 0.5268
Epoch 4/5
26/26 [==============================] - 20s 755ms/step - loss: 0.5371 - accuracy: 0.7530 - val_loss: 0.7156 - val_accuracy: 0.5463
Epoch 5/5
26/26 [==============================] - 20s 757ms/step - loss: 0.4934 - accuracy: 0.7859 - val_loss: 0.4523 - val_accuracy: 0.8049


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.620438,0.659855,0.760976,0.642707
1,1,0.658151,0.625670,0.765854,0.602883
2,2,0.677616,0.592487,0.526829,0.682136
3,3,0.753041,0.536713,0.546342,0.715567
4,4,0.785888,0.493015,0.804878,0.452328


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 20s 786ms/step - loss: 0.6765 - accuracy: 0.5718 - val_loss: 0.6738 - val_accuracy: 0.5366

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 31s 1s/step - loss: 0.6580 - accuracy: 0.6192 - val_loss: 0.6658 - val_accuracy: 0.5659

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 785ms/step - loss: 0.6501 - accuracy: 0.6545 - val_loss: 0.6627 - val_accuracy: 0.5610

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 758ms/step - loss: 0.6453 - accuracy: 0.6436 - val_loss: 0.6595 - val_accuracy: 0.5951

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 755ms/step - loss: 0.6431

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 21s 822ms/step - loss: 0.6742 - accuracy: 0.5596 - val_loss: 0.6576 - val_accuracy: 0.8049
Epoch 2/50
26/26 [==============================] - 21s 796ms/step - loss: 0.6362 - accuracy: 0.6800 - val_loss: 0.6093 - val_accuracy: 0.8780
Epoch 3/50
26/26 [==============================] - 20s 758ms/step - loss: 0.5809 - accuracy: 0.7579 - val_loss: 0.5704 - val_accuracy: 0.7317
Epoch 4/50
26/26 [==============================] - 21s 800ms/step - loss: 0.5264 - accuracy: 0.7908 - val_loss: 0.5130 - val_accuracy: 0.7171
Epoch 5/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4470 - accuracy: 0.8225
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 758ms/step - loss: 0.4468 - accuracy: 0.8224 - val_loss: 0.5513 - val_accuracy: 0.7317
Epoch 6/50
26/26 [==============================] - 20s 765ms/step - loss: 0.3956 - accuracy: 0.8564 - val_loss: 0.3892 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4734), started 0:25:54 ago. (Use '!kill 4734' to kill it.)